In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Polygon, MultiPolygon, Point
import geopandas as gpd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from tqdm import tqdm  # Optional: For progress tracking
from scipy.interpolate import griddata
from statsmodels.tsa.seasonal import seasonal_decompose


## ERA5 Data Documentation
- data documented here: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#heading-Meanratesfluxesandaccumulations

# Loading NetCDF data

In [3]:
reanalysis_nc = xr.open_dataset(r'../datasets/era5/reanalysis_selected.nc')
mun_gdf = gpd.read_file(r'../datasets/municipality_data/municipalities-shapefile-2/concelhos.shp')
reanalysis_df = reanalysis_nc.to_dataframe()

In [5]:
import os
# Path to the folder containing the NetCDF files
folder_path = '../datasets/era5/net_cdf_hourly/'

# List to store individual DataFrames
hourly_dfs = []
quarterly_dfs = []
yearly_dfs = []

# Loop through each file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.nc'):  # Check if the file is a NetCDF file
        file_path = os.path.join(folder_path, file)
        
        # Open the NetCDF file as an xarray dataset
        hourly_ds = xr.open_dataset(file_path)
        # Drop Irrelevant Variables 
        hourly_ds = hourly_ds.drop_vars('sst')
        hourly_ds['tp_1000'] = hourly_ds['tp'] * 1000
        hourly_ds['sp'] = hourly_ds['sp'] * 100

        # Resample Data On Day, Aggregating 
        daily_means = hourly_ds.resample(time='D').mean('time')
        # Resample Data On Month, Aggregating
        monthly_stddevs = daily_means.resample(time='M').std('time')
        
        quarterly_stddevs = monthly_stddevs.resample(time='Q').mean('time')
        avg_year_stddev = monthly_stddevs.resample(time = 'Y').mean('time')
        # Convert the xarray dataset to a DataFrame
        hourly_df = monthly_stddevs.to_dataframe()

        quarterly_df = quarterly_stddevs.to_dataframe()
        yearly_df = avg_year_stddev.to_dataframe()
        # Append the DataFrame to the list
        hourly_dfs.append(hourly_df)
        quarterly_dfs.append(quarterly_df)
        yearly_dfs.append(yearly_df)
# Concatenate all individual DataFrames into one
erA5_std_df = pd.concat(hourly_dfs, axis=0)

erA5_std_df_q = pd.concat(quarterly_dfs, axis=0)
erA5_std_df_y = pd.concat(yearly_dfs, axis=0)


## Quarterly Data Index Reformat

In [6]:
# Convert the time index to 'YYYYQQ' format
quarterly_index = erA5_std_df_q.index.set_levels(erA5_std_df_q.index.levels[2].to_series().dt.to_period('Q').dt.strftime('%YQ%q'), level=2)

# Assign the new index to the dataframe
erA5_std_df_q.index = quarterly_index

erA5_std_df_q = erA5_std_df_q.sort_index(level = ['latitude', 'longitude', 'time'], ascending=True)


### Yearly Data Index Reformat

In [7]:
# Convert the time index to 'YYYYQQ' format
yearly_index = erA5_std_df_y.index.set_levels(erA5_std_df_y.index.levels[2].to_series().dt.to_period('Y').dt.strftime('%Y'), level=2)

# Assign the new index to the dataframe
erA5_std_df_y.index = yearly_index

erA5_std_df_y = erA5_std_df_y.sort_index(level = ['latitude', 'longitude', 'time'], ascending=True)

## Monthly Data Index Reformat

In [8]:
erA5_std_df = erA5_std_df.sort_index(level = ['latitude', 'longitude', 'time'], ascending=True)
# Convert the 'time' level to the first day of each month
erA5_std_df.index.set_levels(levels = erA5_std_df.index.levels[2].strftime('%Y-%m-01'), level=2, inplace=True)
erA5_std_df.index.set_levels(levels = pd.DatetimeIndex(erA5_std_df.index.levels[2]),
    level=2,
    inplace=True
)
erA5_std_df

/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_62077/3176136934.py:3: FutureWarning: inplace is deprecated and will be removed in a future version.
  erA5_std_df.index.set_levels(levels = erA5_std_df.index.levels[2].strftime('%Y-%m-01'), level=2, inplace=True)
/var/folders/6c/vby383sn3xlg0j_c253f0sf40000gn/T/ipykernel_62077/3176136934.py:4: FutureWarning: inplace is deprecated and will be removed in a future version.
  erA5_std_df.index.set_levels(levels = pd.DatetimeIndex(erA5_std_df.index.levels[2]),


t2m         msl      stl2            sp  \
longitude latitude  time                                                       
-9.50     36.919998 1990-01-01  0.750060  623.241089  0.234960  62327.214844   
                    1990-02-01  0.710210  386.238037  0.201591  38623.707031   
                    1990-03-01  0.810310  514.520447  0.243901  51452.785156   
                    1990-04-01  0.780361  675.136414  0.191418  67516.000000   
                    1990-05-01  0.737124  308.730927  0.687092  30875.253906   
...                                  ...         ...       ...           ...   
-6.25     41.919998 2022-08-01  2.802478  251.069351  2.059406  22761.765625   
                    2022-09-01  2.830676  316.210968  2.193844  28969.507812   
                    2022-10-01  2.053990  413.500397  1.834660  41110.738281   
                    2022-11-01  2.218427  548.399475  1.671918  48648.960938   
                    2022-12-01  2.924474  847.471741  1.998029  78596.664062   

                                      tp   tp_1000  
longitude latitude  time                            
-9.50     36.919998 1990-01-01  0.000075  0.075129  
                    1990-02-01  0.000004  0.003643  
                    1990-03-01  0.000483  0.482791  
                    1990-04-01  0.000145  0.145437  
                    1990-05-01  0.000016  0.015668  
...                                  ...       ...  
-6.25     41.919998 2022-08-01  0.000024  0.024456  
                    2022-09-01  0.000131  0.131304  
                    2022-10-01  0.000219  0.218981  
                    2022-11-01  0.000121  0.120530  
                    2022-12-01  0.000229  0.229243  

[116424 rows x 6 columns]

In [9]:
# Create a complete date range from '1990-01-01' to '2022-12-01'
complete_date_range = pd.date_range(start='1990-01-01', end='2022-12-01', freq='MS')

# Identify the missing months
missing_months = complete_date_range.difference(erA5_std_df.index.levels[2].unique())

# Print or use the missing months as needed
print("Missing Months:", missing_months)


Missing Months: DatetimeIndex([], dtype='datetime64[ns]', freq='MS')


## Obtaining historical mean values, and deviations from historical means

In [10]:
reanalysis_df['tp_1000'] = reanalysis_df['tp'] * 1000
mean_monthly_values = reanalysis_df.loc[pd.IndexSlice[:, :, :, '1940-01-01':'1980-12-01']].groupby(['longitude','latitude']).mean()
historical_deviations = reanalysis_df - mean_monthly_values
historical_deviations = historical_deviations.loc[pd.IndexSlice[:, :, :, '1980-06-01':'2023-06-01']]

In [11]:
reanalysis_df

t2m            msl         sst  \
longitude latitude  expver time                                                
-9.50     41.919998 1      1940-01-01  284.716980  101395.757812  286.415314   
                           1940-02-01  285.627289  101372.671875  286.103516   
                           1940-03-01  286.070496  101516.382812  286.168213   
                           1940-04-01  286.509277  101767.781250  287.100616   
                           1940-05-01  287.809998  101585.351562  288.514832   
...                                           ...            ...         ...   
-6.25     36.919998 5      2023-06-01         NaN            NaN         NaN   
                           2023-07-01         NaN            NaN         NaN   
                           2023-08-01         NaN            NaN         NaN   
                           2023-09-01         NaN            NaN         NaN   
                           2023-10-01  295.216125  101550.421875         NaN   

                                             stl2             sp        tp  \
longitude latitude  expver time                                              
-9.50     41.919998 1      1940-01-01  286.425781  101369.875000  0.004636   
                           1940-02-01  286.106018  101346.085938  0.006822   
                           1940-03-01  286.161987  101491.078125  0.002883   
                           1940-04-01  287.081421  101741.781250  0.002486   
                           1940-05-01  288.489960  101558.859375  0.001213   
...                                           ...            ...       ...   
-6.25     36.919998 5      2023-06-01         NaN            NaN       NaN   
                           2023-07-01         NaN            NaN       NaN   
                           2023-08-01         NaN            NaN       NaN   
                           2023-09-01         NaN            NaN       NaN   
                           2023-10-01  297.281799  101351.476562  0.002554   

                                        tp_1000  
longitude latitude  expver time                  
-9.50     41.919998 1      1940-01-01  4.636065  
                           1940-02-01  6.822389  
                           1940-03-01  2.883197  
                           1940-04-01  2.486443  
                           1940-05-01  1.213488  
...                                         ...  
-6.25     36.919998 5      2023-06-01       NaN  
                           2023-07-01       NaN  
                           2023-08-01       NaN  
                           2023-09-01       NaN  
                           2023-10-01  2.553807  

[591528 rows x 7 columns]

### Obtaining the Same for Quarterly Data

#### Reformatting and Resampling Monthly Means to Quarterly Level

In [12]:
reanalysis_df_q = reanalysis_nc.resample(time='Q').mean('time').to_dataframe()
reanalysis_df_q['tp_1000'] = reanalysis_df_q['tp'] * 1000
reanalysis_df_q = reanalysis_df_q.loc[pd.IndexSlice[:, :, 1, :], :].copy()
reanalysis_df_q = reanalysis_df_q.droplevel('expver')
# Convert the time index to 'YYYYQQ' format
quarterly_index_m = reanalysis_df_q.index.set_levels(reanalysis_df_q.index.levels[2].to_series().dt.to_period('Q').dt.strftime('%YQ%q'), level=2)
# Assign the new index to the dataframe
reanalysis_df_q.index = quarterly_index_m

reanalysis_df_q = reanalysis_df_q.sort_index(level = ['latitude', 'longitude', 'time'], ascending=True)


In [13]:
mean_qaurt_values = reanalysis_df_q.sort_index().loc[pd.IndexSlice[:, :,'1940Q1': '1980Q4']].groupby(['longitude','latitude']).mean()
historical_deviations_q = reanalysis_df_q - mean_qaurt_values
historical_deviations_q = historical_deviations_q.loc[pd.IndexSlice[:, :, '1990Q1':'2022Q4']]

### Yearly Average Deviation

In [14]:
reanalysis_df_y = reanalysis_nc.resample(time='Y').mean('time').to_dataframe()
reanalysis_df_y['tp_1000'] = reanalysis_df_y['tp'] * 1000
reanalysis_df_y = reanalysis_df_y.loc[pd.IndexSlice[:, :, 1, :], :].copy()
reanalysis_df_y = reanalysis_df_y.droplevel('expver')
# Convert the time index to 'YYYYQQ' format
yearly_index_m = reanalysis_df_y.index.set_levels(reanalysis_df_y.index.levels[2].to_series().dt.to_period('Y').dt.strftime('%Y'), level=2)
# Assign the new index to the dataframe
reanalysis_df_y.index = yearly_index_m

reanalysis_df_y = reanalysis_df_y.sort_index(level = ['latitude', 'longitude', 'time'], ascending=True)

In [15]:
mean_year_values = reanalysis_df_y.sort_index().loc[pd.IndexSlice[:, :,'1940': '1980']].groupby(['longitude','latitude']).mean()
historical_deviations_y = reanalysis_df_y - mean_year_values
historical_deviations_y = historical_deviations_y.loc[pd.IndexSlice[:, :, '1990':'2022']]

## Assigning Seasonal Dummies

In [16]:
historical_deviations['season'] = historical_deviations.index.get_level_values('time').month.map({
    12: 'winter', 1: 'winter', 2: 'winter',
    3: 'spring', 4: 'spring', 5: 'spring',
    6: 'summer', 7: 'summer', 8: 'summer',
    9: 'autumn', 10: 'autumn', 11: 'autumn'
})

# Convert 'season' column to dummy variables
historical_deviations = pd.get_dummies(historical_deviations, columns=['season'], prefix='season', )
historical_deviations

t2m        msl       sst  \
longitude latitude  expver time                                        
-9.50     36.919998 1      1980-06-01  1.043762  56.390625  0.474884   
                           1980-07-01  1.983398  -4.984375  1.189392   
                           1980-08-01  3.058868  -7.390625  2.082367   
                           1980-09-01  3.151703  55.914062  2.584442   
                           1980-10-01  2.078186  82.593750  2.060059   
...                                         ...        ...       ...   
-6.25     41.919998 5      2023-02-01       NaN        NaN       NaN   
                           2023-03-01       NaN        NaN       NaN   
                           2023-04-01       NaN        NaN       NaN   
                           2023-05-01       NaN        NaN       NaN   
                           2023-06-01       NaN        NaN       NaN   

                                           stl2         sp        tp  \
longitude latitude  expver time                                        
-9.50     36.919998 1      1980-06-01  0.450623  55.835938 -0.001013   
                           1980-07-01  1.173615  -4.992188 -0.001249   
                           1980-08-01  2.067413  -7.234375 -0.000953   
                           1980-09-01  2.590698  55.382812 -0.001041   
                           1980-10-01  2.079742  82.320312 -0.000389   
...                                         ...        ...       ...   
-6.25     41.919998 5      2023-02-01       NaN        NaN       NaN   
                           2023-03-01       NaN        NaN       NaN   
                           2023-04-01       NaN        NaN       NaN   
                           2023-05-01       NaN        NaN       NaN   
                           2023-06-01       NaN        NaN       NaN   

                                        tp_1000  season_autumn  season_spring  \
longitude latitude  expver time                                                 
-9.50     36.919998 1      1980-06-01 -1.013062              0              0   
                           1980-07-01 -1.248605              0              0   
                           1980-08-01 -0.952666              0              0   
                           1980-09-01 -1.040937              1              0   
                           1980-10-01 -0.389128              1              0   
...                                         ...            ...            ...   
-6.25     41.919998 5      2023-02-01       NaN              0              0   
                           2023-03-01       NaN              0              1   
                           2023-04-01       NaN              0              1   
                           2023-05-01       NaN              0              1   
                           2023-06-01       NaN              0              0   

                                       season_summer  season_winter  
longitude latitude  expver time                                      
-9.50     36.919998 1      1980-06-01              1              0  
                           1980-07-01              1              0  
                           1980-08-01              1              0  
                           1980-09-01              0              0  
                           1980-10-01              0              0  
...                                              ...            ...  
-6.25     41.919998 5      2023-02-01              0              1  
                           2023-03-01              0              0  
                           2023-04-01              0              0  
                           2023-05-01              0              0  
                           2023-06-01              1              0  

[303996 rows x 11 columns]

In [17]:
erA5_std_df['season'] = erA5_std_df.index.get_level_values('time').month.map({
    12: 'winter', 1: 'winter', 2: 'winter',
    3: 'spring', 4: 'spring', 5: 'spring',
    6: 'summer', 7: 'summer', 8: 'summer',
    9: 'autumn', 10: 'autumn', 11: 'autumn'
})

# Convert 'season' column to dummy variables
erA5_std_df = pd.get_dummies(erA5_std_df, columns=['season'], prefix='season', )

# Polygons

In [18]:
class PolygonConversion:

    @staticmethod
    def fetch_geo_polygon(latitude: float, longitude: float, lat_grid_resolution: float, lon_grid_resolution: float) -> Polygon:
        """Create a Polygon based on latitude, longitude, and resolution.

        Example ::
            * - . - *
            |       |
            .   •   .
            |       |
            * - . - *
        In order to create the polygon, we require the `*` point as indicated in the above example.
        To determine the position of the `*` point, we find the `.` point.
        The `get_lat_lon_range` function gives the `.` point and `bound_point` gives the `*` point.
            """        # Calculate the half-size of the bounding box
        half_size_lon = lon_grid_resolution / 2
        half_size_lat = lat_grid_resolution / 2
        
        # Calculate the bound points
        lower_left = (longitude - half_size_lon, latitude - half_size_lat)
        upper_left = (longitude - half_size_lon, latitude + half_size_lat)
        upper_right = (longitude + half_size_lon, latitude + half_size_lat)
        lower_right = (longitude + half_size_lon, latitude - half_size_lat)
        
        polygon = Polygon([lower_left, upper_left, upper_right, lower_right])
        return polygon



## Creating Points from Lat,and Lon and Polygons centered on Points

In [19]:
polygon_converter = PolygonConversion()

# Assume your dataset is named 'ds' (replace it with the actual name)
# Accessing latitude and longitude from the dataset
lats = historical_deviations.index.get_level_values('latitude').values
lons = historical_deviations.index.get_level_values('longitude').values

# Spatial resolution of your data
lat_resolution = 0.25
lon_resolution = 0.25

# Create a DataFrame to store latitudes and longitudes
df_points = pd.DataFrame({
    'lat': lats,
    'lon': lons
})

geometry = [Point(lon, lat) for lon, lat in zip(lons, lats)]
gdf_points = gpd.GeoDataFrame(geometry = geometry, crs="EPSG:4326", index=historical_deviations.index)
gdf_points.rename(columns = {'geometry' : 'points_geometry'}, inplace = True)
# Create a GeoDataFrame with Polygon geometries
polygons = []
for _, row in tqdm(df_points.iterrows(), total=len(df_points), desc="Creating Polygons"):
    lat, lon = row['lat'], row['lon']
    polygon = polygon_converter.fetch_geo_polygon(lat, lon, lat_resolution, lon_resolution)
    polygons.append(polygon)

# Create a GeoDataFrame with the constructed polygons
gdf_polygons = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")
gdf_polygons = gdf_polygons.set_index(historical_deviations.index)

Creating Polygons: 100%|██████████| 303996/303996 [00:06<00:00, 43751.29it/s]


In [20]:
polygon_converter = PolygonConversion()

# Assume your dataset is named 'ds' (replace it with the actual name)
# Accessing latitude and longitude from the dataset
lats_std = erA5_std_df.index.get_level_values('latitude').values
lons_std = erA5_std_df.index.get_level_values('longitude').values

# Spatial resolution of your data
lat_resolution = 0.25
lon_resolution = 0.25

# Create a DataFrame to store latitudes and longitudes
df_points_std = pd.DataFrame({
    'lat': lats_std,
    'lon': lons_std
})
geometry_std = [Point(lon, lat) for lon, lat in zip(lons_std, lats_std)]
gdf_points_std = gpd.GeoDataFrame(geometry = geometry_std, crs="EPSG:4326", index=erA5_std_df.index)
gdf_points_std.rename(columns = {'geometry' : 'points_geometry'}, inplace = True)

# Create a GeoDataFrame with Polygon geometries
polygons_std = []
for _, row in tqdm(df_points_std.iterrows(), total=len(df_points_std), desc="Creating Polygons"):
    lat, lon = row['lat'], row['lon']
    polygon = polygon_converter.fetch_geo_polygon(lat, lon, lat_resolution, lon_resolution)
    polygons_std.append(polygon)

# Create a GeoDataFrame with the constructed polygons
gdf_polygons_std = gpd.GeoDataFrame(geometry=polygons_std, crs="EPSG:4326")
gdf_polygons_std = gdf_polygons_std.set_index(erA5_std_df.index)

Creating Polygons: 100%|██████████| 116424/116424 [00:02<00:00, 42616.79it/s]


## Merging Polygons with Renalysis Data

In [21]:

# Extract latitude and longitude from the MultiIndex DataFrame
latitudes = historical_deviations.index.get_level_values('latitude').values
longitudes = historical_deviations.index.get_level_values('longitude').values


# Merge the GeoDataFrame into the MultiIndex DataFrame
merged_df = pd.concat([historical_deviations, gdf_points, gdf_polygons], axis=1)


In [22]:

# Extract latitude and longitude from the MultiIndex DataFrame
latitudes_std = erA5_std_df.index.get_level_values('latitude').values
longitudes_std = erA5_std_df.index.get_level_values('longitude').values


# Merge the GeoDataFrame into the MultiIndex DataFrame
merged_std_df = pd.concat([erA5_std_df, gdf_points_std, gdf_polygons_std], axis=1)


## ExpVer Explanation

In [23]:
nan_counts = historical_deviations.groupby('expver').apply(lambda x: x.isna().sum())
nan_counts

,t2m,msl,sst,stl2,sp,tp,tp_1000,season_autumn,season_spring,season_summer,season_winter
expver,,,,,,,,,,,
1,0,0,118393,0,0,0,0,0,0,0,0
5,151998,151998,151998,151998,151998,151998,151998,0,0,0,0


The latest 3 months in this dataset are made available through ERA5T, which might be slightly different to ERA5. In the downloaded file, an extra dimenions ‘expver’ indicates which data is ERA5 (expver = 1) and which is ERA5T (expver = 5).

In [24]:
# Select rows where expver is 1
merged_df_1 = merged_df.loc[pd.IndexSlice[:, :, 1, :], :].copy()
# Select rows where expver is 5
merged_df_5 = merged_df.loc[pd.IndexSlice[:, :, 5, :], :].copy()


## Dropping NaN Trend

In [25]:
merged_df_1 = merged_df_1.loc[pd.IndexSlice[:, :, :, '1990-01-01':'2022-12-01']]

# Convert to Xarray and Save NetCDF

In [26]:
historical_deviations_ds = merged_df_1[['t2m', 'msl', 'sst', 'stl2', 'sp', 'tp','tp_1000', 'season_autumn',
       'season_spring', 'season_summer', 'season_winter']].to_xarray()
std_ds = merged_std_df[['t2m', 'msl', 'stl2', 'sp', 'tp', 'tp_1000','season_autumn', 'season_spring',
       'season_summer', 'season_winter']].to_xarray()

In [28]:
historical_deviations_ds.to_netcdf(r'../datasets/era5/historical_deviations.nc')

In [29]:
std_ds.to_netcdf(r'../datasets/era5/std_ds.nc')

### Quarterly Data

In [30]:
historical_deviations_q.to_xarray().to_netcdf(r'../datasets/era5/historical_deviations_qaurt.nc')

erA5_std_df_q.to_xarray().to_netcdf(r'../datasets/era5/std_ds_qaurt.nc')

### Yearly Data

In [31]:
historical_deviations_y.to_xarray().to_netcdf(r'../datasets/era5/historical_deviations_y.nc')

erA5_std_df_y.to_xarray().to_netcdf(r'../datasets/era5/std_ds_yearly.nc')